In [ ]:
import pandas as pd

# Collab Filtering Recommender



In [ ]:
similarity_df = pd.read_csv('item_similarity.csv', index_col = 0)
similarity_df

In [ ]:
def get_similar_songs(songURL):
    similar_score = similarity_df[songURL]*0.5
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score

In [10]:
def get_similar_playlist_with_names(songs):
    similar_songs = pd.DataFrame()
    for song in songs:
        similar_songs = similar_songs.append(get_similar_songs(song), ignore_index=True)
    similar_songs = similar_songs.sum().sort_values(ascending=False)
    return similar_songs

get_similar_playlist(['spotify:track:7zsw78LtXUD7JfEwH64HK2', 'spotify:track:7zBPzAjKAqQpcv8F8GCq5s']) 


spotify:track:7zBPzAjKAqQpcv8F8GCq5s    0.496708
spotify:track:7zsw78LtXUD7JfEwH64HK2    0.496708
spotify:track:0zstgBrV1t1g6n4jHrUVBY    0.347132
spotify:track:4HGIPyqDxSf863tPOwXiLJ    0.324174
spotify:track:0Q4a3PdGEME9w8Jgqa0Gf3    0.309010
spotify:track:1lOSxJNCLvWm2bYaTcTSmK    0.309010
spotify:track:6oYkwjI1TKP9D0Y9II1GT7    0.304257
spotify:track:2AILbz83cBnrAMAG06rZts    0.298751
spotify:track:2rJFFUEl1LURkV0b0OARXx    0.298077
spotify:track:0I8mZRtlFGOqZayJbEAdxL    0.294661
spotify:track:7tUSJY4nsDBJTjd1UXKRsT    0.286436
spotify:track:0CKmN3Wwk8W4zjU0pqq2cv    0.282786
spotify:track:3uZIfWx5ridUBQevmgHDUt    0.282576
spotify:track:7sYaJXMEvq3q91jJNm8I7I    0.282576
spotify:track:6btdYzQ8eZFBrOlUKVHuz0    0.277328
spotify:track:3uVkugZz6yCTw5Z8sDI19F    0.262673
spotify:track:6hhVS4jyOPioRJaY1zVRHj    0.262066
spotify:track:1HSJMLtNeDryJN76Tu5Ahf    0.259758
spotify:track:5E8yddtSG3sH4hw2SfwcgW    0.258483
spotify:track:6mDxu0xwhv5tn1oMTNUypu    0.257292
spotify:track:70b5Sq

In [ ]:
def get_similar_playlist(songs):
    similar_songs = pd.DataFrame()
    for song in songs:
        similar_songs = similar_songs.append(get_similar_songs(song), ignore_index=True)
    similar_songs = similar_songs.sum().sort_values(ascending=False)
    return similar_songs

get_similar_playlist(['spotify:track:7zsw78LtXUD7JfEwH64HK2', 'spotify:track:7zBPzAjKAqQpcv8F8GCq5s'])
